In [1]:
import pandas as pd
import numpy as np
import requests
import re 
from geopy.geocoders import Nominatim
from bs4 import BeautifulSoup


In [2]:
URL = "https://en.wikipedia.org/wiki/List_of_London_boroughs"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')

df_list = []
# print(soup)
for items in soup.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data1 = data[0]
    data8 = data[8]    
    try:
        borough_name = data1.get_text()
        borough_name = borough_name.split('[')
        borough_name = borough_name[0]
        borough_name = borough_name.strip()
        
        ll = data8.get_text()
        ll = ll.split('/')
        lat_long = ll[2]
        lat_long = lat_long.split('(')
        lat_long = lat_long[0]
        lat_long = lat_long.split(';')
        latitude = lat_long[0]
        latitude = latitude.strip()
        longitude = lat_long[1]
        longitude = longitude.strip()
        longitude = longitude.replace(u'\ufeff', '')
        latitude = float(latitude)
        longitude = float(longitude)

#       Append the borough name, latitude and logitude in a list
        df_list.append((borough_name, latitude, longitude))
    except IndexError:pass

In [3]:
df_boroughs = pd.DataFrame(df_list, columns=['Borough', 'Latitude' , 'Longitude'])
df_boroughs.dtypes

Borough       object
Latitude     float64
Longitude    float64
dtype: object

In [4]:
df_boroughs.head()

,Borough,Latitude,Longitude
0,Barking and Dagenham,51.5607,0.1557
1,Barnet,51.6252,-0.1517
2,Bexley,51.4549,0.1505
3,Brent,51.5588,-0.2817
4,Bromley,51.4039,0.0198


In [5]:
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
# import k-means from clustering stage
from sklearn.cluster import KMeans

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# install and import folium library 
!pip -q install folium
import folium 

# import k-means from clustering stage
from sklearn.cluster import KMeans

In [6]:
#co-ordinates of london city
from geopy.geocoders import Nominatim 
address = 'London, UK'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of London City are 51.5073219, -0.1276474.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [9]:
#map of london
map_london=folium.Map(Location=(latitude,longitude),zoom_start=10)


In [8]:
#markers
for lat, lng, borough in zip(df_boroughs['Latitude'], df_boroughs['Longitude'], df_boroughs['Borough']):
    label = '{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
fill_opacity=0.7).add_to(map_london)  
map_london

Since I am new to the area,I want to stay in cheaper place with basic amenities,bus stops and train stations around me.The rent distribution data of the London city was taken from following link:
    
https://www.london.gov.uk/what-we-do/housing-and-land/improving-private-rented-sector/london-rents-map#acc-i-48689

I am choosing Enfield area,since the rent range is in my budget and it is nearest to the downtown london of all the budgeted rent areas!

In [10]:
# Defining four square credentials 
    
CLIENT_ID = 'JUDOSIB3H5LAUMCG13B52RSIVM1L4O2QREH3ADMQDVUAYRS3' # Foursquare ID
CLIENT_SECRET = 'YYBKD0L1IDV4D54UH5O1JT3UPNXOB01DTLKGXN4QOJ44SODS' #  Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:Soumya')
print('CLIENT_ID:'  + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET )  

Your credentails:Soumya
CLIENT_ID:JUDOSIB3H5LAUMCG13B52RSIVM1L4O2QREH3ADMQDVUAYRS3
CLIENT_SECRET:YYBKD0L1IDV4D54UH5O1JT3UPNXOB01DTLKGXN4QOJ44SODS


In [11]:
from urllib.request import urlopen
import re
URL = "https://en.wikipedia.org/wiki/List_of_areas_of_London"
res = requests.get(URL).text
soup = BeautifulSoup(res,'lxml')

codes = []
areas_list = []
href_links_list = []
for items in soup.find('table', class_= 'wikitable sortable').find_all('tr')[1::]:
    data = items.find_all(['td'])
    data0 = data[0]
    area_name = data0.text

    data1 = data[1]
    data1 = data1.text
    borough = data1.split('[')
    borough_name = borough[0]
    data5 = data[5]
    code = data5.text
    code = code.strip()
    
    if borough_name == 'Enfield':
        codes.append(code)
        areas_list.append((borough_name,area_name,code))

                
for link in soup.findAll('a', attrs={'href': re.compile("^https://tools.wmflabs.org")}):
            htext = link.text
            if htext in codes:
                hlink = link.get('href')
                href_links_list.append((htext, hlink))

In [12]:
df_areas = pd.DataFrame(areas_list, columns=['Borough', 'Area', 'Code'])


In [13]:


df_areas

,Borough,Area,Code
0,Enfield,Arnos Grove,TQ295925
1,Enfield,Botany Bay,TQ297992
2,Enfield,Brimsdown,TQ365975
3,Enfield,Bulls Cross,TQ342994
4,Enfield,Crews Hill,TQ315995
5,Enfield,Edmonton,TQ335925
6,Enfield,Enfield Highway,TQ355975
7,Enfield,Enfield Lock,TQ365985
8,Enfield,Enfield Town,TQ325965
9,Enfield,Enfield Wash,TQ355985


In [14]:
df_links = pd.DataFrame(href_links_list, columns=['Code','href'])
                                                 

In [15]:
df_links

,Code,href
0,TQ295925,https://tools.wmflabs.org/geohack/en/51.616437...
1,TQ297992,https://tools.wmflabs.org/geohack/en/51.676600...
2,TQ365975,https://tools.wmflabs.org/geohack/en/51.659715...
3,TQ342994,https://tools.wmflabs.org/geohack/en/51.677341...
4,TQ315995,https://tools.wmflabs.org/geohack/en/51.678878...
5,TQ335925,https://tools.wmflabs.org/geohack/en/51.615504...
6,TQ355975,https://tools.wmflabs.org/geohack/en/51.659956...
7,TQ365985,https://tools.wmflabs.org/geohack/en/51.668700...
8,TQ325965,https://tools.wmflabs.org/geohack/en/51.651685...
9,TQ355985,https://tools.wmflabs.org/geohack/en/51.668942...


In [16]:
cols=df_links.columns.difference(df_areas.columns)


In [17]:

df_links.shape

(18, 2)

In [18]:
df_final=pd.concat([df_areas, df_links[cols]], axis=1)


In [19]:
df_final


,Borough,Area,Code,href
0,Enfield,Arnos Grove,TQ295925,https://tools.wmflabs.org/geohack/en/51.616437...
1,Enfield,Botany Bay,TQ297992,https://tools.wmflabs.org/geohack/en/51.676600...
2,Enfield,Brimsdown,TQ365975,https://tools.wmflabs.org/geohack/en/51.659715...
3,Enfield,Bulls Cross,TQ342994,https://tools.wmflabs.org/geohack/en/51.677341...
4,Enfield,Crews Hill,TQ315995,https://tools.wmflabs.org/geohack/en/51.678878...
5,Enfield,Edmonton,TQ335925,https://tools.wmflabs.org/geohack/en/51.615504...
6,Enfield,Enfield Highway,TQ355975,https://tools.wmflabs.org/geohack/en/51.659956...
7,Enfield,Enfield Lock,TQ365985,https://tools.wmflabs.org/geohack/en/51.668700...
8,Enfield,Enfield Town,TQ325965,https://tools.wmflabs.org/geohack/en/51.651685...
9,Enfield,Enfield Wash,TQ355985,https://tools.wmflabs.org/geohack/en/51.668942...


In [20]:
df_final = df_final.dropna(how='any')

In [21]:
df_final

,Borough,Area,Code,href
0,Enfield,Arnos Grove,TQ295925,https://tools.wmflabs.org/geohack/en/51.616437...
1,Enfield,Botany Bay,TQ297992,https://tools.wmflabs.org/geohack/en/51.676600...
2,Enfield,Brimsdown,TQ365975,https://tools.wmflabs.org/geohack/en/51.659715...
3,Enfield,Bulls Cross,TQ342994,https://tools.wmflabs.org/geohack/en/51.677341...
4,Enfield,Crews Hill,TQ315995,https://tools.wmflabs.org/geohack/en/51.678878...
5,Enfield,Edmonton,TQ335925,https://tools.wmflabs.org/geohack/en/51.615504...
6,Enfield,Enfield Highway,TQ355975,https://tools.wmflabs.org/geohack/en/51.659956...
7,Enfield,Enfield Lock,TQ365985,https://tools.wmflabs.org/geohack/en/51.668700...
8,Enfield,Enfield Town,TQ325965,https://tools.wmflabs.org/geohack/en/51.651685...
9,Enfield,Enfield Wash,TQ355985,https://tools.wmflabs.org/geohack/en/51.668942...


In [22]:
geo_codes = []
for row in df_final.itertuples():
    url = row.href
    code = row.Code
    res = requests.get(url).text
    soup1 = BeautifulSoup(res,'lxml')
    
    for lat in soup1.find('span',{'class':'latitude'}):
        latitude = lat
        latitude = float(latitude)
            
    for long in soup1.find('span',{'class':'longitude'}):    
        longitude = long
        longitude = float(longitude)
        
    geo_codes.append((code, latitude, longitude))

print(geo_codes)
 

[('TQ295925', 51.616438, -0.130954), ('TQ297992', 51.676601, -0.125585), ('TQ365975', 51.659715, -0.027963), ('TQ342994', 51.677342, -0.060466), ('TQ315995', 51.678879, -0.099455), ('TQ335925', 51.615504, -0.073214), ('TQ355975', 51.659957, -0.042412), ('TQ365985', 51.668701, -0.027573), ('TQ325965', 51.651685, -0.086136), ('TQ355985', 51.668943, -0.042024), ('TQ361993', 51.675987, -0.033042), ('TQ313962', 51.649271, -0.103584), ('TQ265975', 51.662053, -0.172455), ('TQ305935', 51.616438, -0.130954), ('TQ355955', 51.625193, -0.116146), ('TQ296942', 51.641985, -0.043186), ('TQ315945', 51.631692, -0.128882)]


In [23]:
df_geo_codes = pd.DataFrame(geo_codes, columns=['Code','Latitude','Longitude'])

In [67]:
df_geo_codes.head(18)

,Code,Latitude,Longitude
0,TQ295925,51.616438,-0.130954
1,TQ297992,51.676601,-0.125585
2,TQ365975,51.659715,-0.027963
3,TQ342994,51.677342,-0.060466
4,TQ315995,51.678879,-0.099455
5,TQ335925,51.615504,-0.073214
6,TQ355975,51.659957,-0.042412
7,TQ365985,51.668701,-0.027573
8,TQ325965,51.651685,-0.086136
9,TQ355985,51.668943,-0.042024


In [68]:
cols = df_geo_codes.columns.difference(df_areas.columns)
df_final1 = pd.concat([df_areas, df_geo_codes[cols]], axis=1)

In [69]:
df_final1.rename(columns={'Area':'Neighborhood'},inplace=True)
df_final1.head()


,Borough,Neighborhood,Code,Latitude,Longitude
0,Enfield,Arnos Grove,TQ295925,51.616438,-0.130954
1,Enfield,Botany Bay,TQ297992,51.676601,-0.125585
2,Enfield,Brimsdown,TQ365975,51.659715,-0.027963
3,Enfield,Bulls Cross,TQ342994,51.677342,-0.060466
4,Enfield,Crews Hill,TQ315995,51.678879,-0.099455


In [28]:
#geo coordinates of Enfield

from geopy.geocoders import Nominatim 
address = 'Enfield, UK'
geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Enfield are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Enfield are 51.6520851, -0.0810175.


/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:5: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.


In [29]:
map_Enfield= folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, label in zip(df_final1['Latitude'], df_final1['Longitude'], df_final1['Area']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7).add_to(map_Enfield)  
    
map_Enfield

In [30]:
#top 100 venues around Enfield 
LIMIT = 100 
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore? &client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
   longitude, 
    radius, 
    LIMIT)
url 

'https://api.foursquare.com/v2/venues/explore? &client_id=JUDOSIB3H5LAUMCG13B52RSIVM1L4O2QREH3ADMQDVUAYRS3&client_secret=YYBKD0L1IDV4D54UH5O1JT3UPNXOB01DTLKGXN4QOJ44SODS&v=20180605&ll=51.6520851,-0.0810175&radius=500&limit=100'

In [31]:
neighbourhood_json = requests.get(url).json()
neighbourhood_json

{'meta': {'code': 200, 'requestId': '5deb53a740a7ea450cd80c50'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Grange',
  'headerFullLocation': 'Grange, London',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 62,
  'suggestedBounds': {'ne': {'lat': 51.6565851045,
    'lng': -0.07377806035278722},
   'sw': {'lat': 51.6475850955, 'lng': -0.08825693964721279}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4c84fc6e2f1c236af8445043',
       'name': 'Goodlooking Optics',
       'location': {'address': '43 Church Street',
        'crossStreet': 'Church Street',
        'lat': 51.65222318701091,
        'lng': -0.07948966142976298,
        'labeledLatLngs': [{'label'

In [32]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

In [33]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    requests.get(url).json() 



In [34]:
venues = neighbourhood_json['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(20)

,name,categories,lat,lng
0,Goodlooking Optics,Optical Shop,51.652223,-0.079490
1,PizzaExpress,Pizza Place,51.652475,-0.080832
2,Waitrose & Partners,Supermarket,51.651602,-0.084114
3,Enfield Town Park,Park,51.649998,-0.083855
4,Caffè Nero,Coffee Shop,51.652061,-0.081949
5,Enfield Town Railway Station (ENF),Train Station,51.652049,-0.079013
6,Aksular Restaurant Enfield Town,Turkish Restaurant,51.652533,-0.080654
7,Nando's,Portuguese Restaurant,51.652141,-0.081041
8,Fish&Chips@,Fish & Chips Shop,51.648624,-0.079847
9,Waterstones,Bookstore,51.652179,-0.083695


In [35]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

62 venues were returned by Foursquare.


In [36]:
#creating a function to repeat the process for all neighbourhoods in enfield
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
       # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [37]:
Enfield_venues = getNearbyVenues(names=df_final1['Area'],
                                   latitudes=df_final1['Latitude'],
                                 longitudes=df_final1['Longitude']
                                  )

Arnos Grove
Botany Bay
Brimsdown
Bulls Cross
Crews Hill
Edmonton
Enfield Highway
Enfield Lock
Enfield Town
Enfield Wash
Freezywater
Grange Park
Hadley Wood
Palmers Green
Ponders End
Southgate
Winchmore Hill


In [38]:
print(Enfield_venues.shape)
Enfield_venues.head()

(188, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Arnos Grove,51.616438,-0.130954,The Arnos Arms,51.616044,-0.132136,Beer Bar
1,Arnos Grove,51.616438,-0.130954,Arnos Park,51.618336,-0.130720,Park
2,Arnos Grove,51.616438,-0.130954,Sainsbury's Local,51.616347,-0.135881,Grocery Store
3,Arnos Grove,51.616438,-0.130954,Arnos Grove London Underground Station,51.616396,-0.133103,Metro Station
4,Arnos Grove,51.616438,-0.130954,Bus Stop A,51.615947,-0.132954,Bus Stop


In [39]:
Enfield_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arnos Grove,6,6,6,6,6,6
Botany Bay,2,2,2,2,2,2
Brimsdown,4,4,4,4,4,4
Bulls Cross,5,5,5,5,5,5
Crews Hill,5,5,5,5,5,5
Edmonton,7,7,7,7,7,7
Enfield Highway,8,8,8,8,8,8
Enfield Lock,5,5,5,5,5,5
Enfield Town,70,70,70,70,70,70


In [40]:
#Analyzing each neighbourhood
# one hot encoding
Enfield_onehot = pd.get_dummies(Enfield_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Enfield_onehot['Neighborhood'] = Enfield_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Enfield_onehot.columns[-1]] + list(Enfield_onehot.columns[:-1])
Enfield_onehot = Enfield_onehot[fixed_columns]

Enfield_onehot.head(18)

,Neighborhood,Bakery,Bar,Beer Bar,Bookstore,Breakfast Spot,Bus Station,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,Garden,Garden Center,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Lebanese Restaurant,Men's Store,Metro Station,Mobile Phone Shop,Museum,Music Venue,Optical Shop,Outdoor Supply Store,Park,Pharmacy,Pizza Place,Platform,Playground,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Soccer Field,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stationery Store,Steakhouse,Supermarket,Tennis Court,Theater,Trail,Train Station,Tunnel,Turkish Restaurant,Video Game Store,Women's Store
0,Arnos Grove,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Arnos Grove,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,Arnos Grove,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,Arnos Grove,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,Arnos Grove,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,Arnos Grove,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,Botany Bay,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,Botany Bay,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0
8,Brimsdown,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,Brimsdown,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [41]:
Enfield_grouped = Enfield_onehot.groupby('Neighborhood').mean().reset_index()
Enfield_grouped.head(18)

,Neighborhood,Bakery,Bar,Beer Bar,Bookstore,Breakfast Spot,Bus Station,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,Garden,Garden Center,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Lebanese Restaurant,Men's Store,Metro Station,Mobile Phone Shop,Museum,Music Venue,Optical Shop,Outdoor Supply Store,Park,Pharmacy,Pizza Place,Platform,Playground,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Soccer Field,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stationery Store,Steakhouse,Supermarket,Tennis Court,Theater,Trail,Train Station,Tunnel,Turkish Restaurant,Video Game Store,Women's Store
0,Arnos Grove,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.0,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
1,Botany Bay,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
2,Brimsdown,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000,0.000000,0.000000
3,Bulls Cross,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.20,0.000000,0.00,0.000000,0.000000,0.000000
4,Crews Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.00,0.000000

In [42]:
#Top 5 venues in each neighbourhood
num_top_venues = 10
for hood in Enfield_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Enfield_grouped[Enfield_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arnos Grove----
                venue  freq
0            Beer Bar  0.17
1       Metro Station  0.17
2            Bus Stop  0.17
3                Park  0.17
4  Chinese Restaurant  0.17
5       Grocery Store  0.17
6              Bakery  0.00
7            Platform  0.00
8          Restaurant  0.00
9                 Pub  0.00


----Botany Bay----
                   venue  freq
0                    Pub   0.5
1            Sports Club   0.5
2                 Bakery   0.0
3             Shoe Store   0.0
4             Restaurant   0.0
5  Portuguese Restaurant   0.0
6             Playground   0.0
7               Platform   0.0
8            Pizza Place   0.0
9               Pharmacy   0.0


----Brimsdown----
                   venue  freq
0   Fast Food Restaurant  0.25
1                    Gym  0.25
2         Breakfast Spot  0.25
3                   Café  0.25
4         Sandwich Place  0.00
5             Restaurant  0.00
6                    Pub  0.00
7  Portuguese Restaurant  0.00
8          

In [52]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [53]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Enfield_grouped['Neighborhood']

for ind in np.arange(Enfield_grouped.shape[0]):
      neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Enfield_grouped.iloc[ind, :], num_top_venues)

In [54]:
neighborhoods_venues_sorted.head

<bound method NDFrame.head of        Neighborhood 1st Most Common Venue 2nd Most Common Venue  \
0       Arnos Grove         Grocery Store              Beer Bar   
1        Botany Bay                   Pub           Sports Club   
2         Brimsdown        Breakfast Spot                   Gym   
3       Bulls Cross                 Trail          Soccer Field   
4        Crews Hill         Garden Center                   Pub   
5          Edmonton    Turkish Restaurant         Train Station   
6   Enfield Highway         Grocery Store     Indian Restaurant   
7      Enfield Lock     Fish & Chips Shop         Train Station   
8      Enfield Town                  Café           Coffee Shop   
9      Enfield Wash                Bakery         Grocery Store   
10      Freezywater                  Café            Shoe Store   
11      Grange Park           Golf Course           Cheese Shop   
12      Hadley Wood                Tunnel                 Trail   
13    Palmers Green         Groc

In [55]:
Enfield_grouped.head()

,Neighborhood,Bakery,Bar,Beer Bar,Bookstore,Breakfast Spot,Bus Station,Bus Stop,Café,Cheese Shop,Chinese Restaurant,Clothing Store,Coffee Shop,Construction & Landscaping,Convenience Store,Deli / Bodega,Department Store,Dessert Shop,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,Garden,Garden Center,Gas Station,Gift Shop,Golf Course,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hotel,Ice Cream Shop,Indian Restaurant,Irish Pub,Italian Restaurant,Lebanese Restaurant,Men's Store,Metro Station,Mobile Phone Shop,Museum,Music Venue,Optical Shop,Outdoor Supply Store,Park,Pharmacy,Pizza Place,Platform,Playground,Portuguese Restaurant,Pub,Restaurant,Sandwich Place,Shoe Store,Shopping Mall,Smoke Shop,Soccer Field,Spanish Restaurant,Sporting Goods Shop,Sports Club,Stationery Store,Steakhouse,Supermarket,Tennis Court,Theater,Trail,Train Station,Tunnel,Turkish Restaurant,Video Game Store,Women's Store
0,Arnos Grove,0.0,0.0,0.166667,0.0,0.00,0.0,0.166667,0.00,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Botany Bay,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Brimsdown,0.0,0.0,0.000000,0.0,0.25,0.0,0.000000,0.25,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.25,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bulls Cross,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.200000,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0
4,Crews Hill,0.0,0.0,0.000000,0.0,0.00,0.0,0.000000,0.00,0.0,0.000000,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.000000,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.2,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [57]:
# Clustering the neighbourhood

#set number of clusters
kclusters = 5
Enfield_grouped_clustering = Enfield_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Enfield_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 0, 4, 1, 4, 1, 4, 0, 0], dtype=int32)

In [58]:
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=5, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=0, tol=0.0001, verbose=0)

In [71]:

Enfield_merged = df_final1
# add clustering labels
Enfield_merged['Cluster Labels'] = kmeans.labels_

# merge Neighborhoods dataframe with Newham borough dataframe to add latitude/longitude for each neighborhood
Enfield_merged = Enfield_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Enfield_merged.head() # check the last columns!

,Borough,Neighborhood,Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Enfield,Arnos Grove,TQ295925,51.616438,-0.130954,1,Grocery Store,Beer Bar,Metro Station,Bus Stop,Park,Chinese Restaurant,Golf Course,Garden,Garden Center,Gas Station
1,Enfield,Botany Bay,TQ297992,51.676601,-0.125585,3,Pub,Sports Club,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,Garden,Garden Center,Gas Station,Women's Store
2,Enfield,Brimsdown,TQ365975,51.659715,-0.027963,0,Breakfast Spot,Gym,Café,Fast Food Restaurant,Women's Store,Golf Course,French Restaurant,Garden,Garden Center,Gas Station
3,Enfield,Bulls Cross,TQ342994,51.677342,-0.060466,4,Trail,Soccer Field,Garden,Park,Pub,Women's Store,Garden Center,Fast Food Restaurant,Fish & Chips Shop,French Restaurant
4,Enfield,Crews Hill,TQ315995,51.678879,-0.099455,1,Garden Center,Pub,Construction & Landscaping,Museum,Gift Shop,Fish & Chips Shop,French Restaurant,Garden,Gas Station,Women's Store


In [72]:
#Analyzing clusters
Enfield_merged.loc[Enfield_merged['Cluster Labels'] == 0, Enfield_merged.columns[[1] + list(range(5, Enfield_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Brimsdown,0,Breakfast Spot,Gym,Café,Fast Food Restaurant,Women's Store,Golf Course,French Restaurant,Garden,Garden Center,Gas Station
8,Enfield Town,0,Café,Coffee Shop,Pub,Clothing Store,Mobile Phone Shop,Optical Shop,Women's Store,Shopping Mall,Bookstore,Department Store
9,Enfield Wash,0,Bakery,Grocery Store,Supermarket,Dessert Shop,Pizza Place,Gas Station,Café,Gym / Fitness Center,Gym Pool,Bar
10,Freezywater,0,Café,Shoe Store,Deli / Bodega,Grocery Store,Pizza Place,Coffee Shop,Women's Store,Gas Station,French Restaurant,Garden
14,Ponders End,0,Pub,Café,Hotel,French Restaurant,Tennis Court,Steakhouse,Gift Shop,Fast Food Restaurant,Fish & Chips Shop,Garden
15,Southgate,0,Café,Breakfast Spot,Playground,Fast Food Restaurant,Supermarket,Pub,Turkish Restaurant,Gym,Grocery Store,Gym / Fitness Center
16,Winchmore Hill,0,Fast Food Restaurant,Café,Gym / Fitness Center,Coffee Shop,Italian Restaurant,Grocery Store,Ice Cream Shop,Greek Restaurant,Lebanese Restaurant,Electronics Store


In [73]:
Enfield_merged.loc[Enfield_merged['Cluster Labels'] == 1, Enfield_merged.columns[[1] + list(range(5, Enfield_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arnos Grove,1,Grocery Store,Beer Bar,Metro Station,Bus Stop,Park,Chinese Restaurant,Golf Course,Garden,Garden Center,Gas Station
4,Crews Hill,1,Garden Center,Pub,Construction & Landscaping,Museum,Gift Shop,Fish & Chips Shop,French Restaurant,Garden,Gas Station,Women's Store
6,Enfield Highway,1,Grocery Store,Indian Restaurant,Turkish Restaurant,Fish & Chips Shop,Pharmacy,Park,Pub,Gas Station,French Restaurant,Garden
13,Palmers Green,1,Grocery Store,Beer Bar,Metro Station,Bus Stop,Park,Chinese Restaurant,Golf Course,Garden,Garden Center,Gas Station


In [74]:
Enfield_merged.loc[Enfield_merged['Cluster Labels'] == 2, Enfield_merged.columns[[1] + list(range(5, Enfield_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Grange Park,2,Golf Course,Cheese Shop,Electronics Store,Fish & Chips Shop,French Restaurant,Garden,Garden Center,Gas Station,Gift Shop,Women's Store


In [75]:
Enfield_merged.loc[Enfield_merged['Cluster Labels'] == 3, Enfield_merged.columns[[1] + list(range(5, Enfield_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Botany Bay,3,Pub,Sports Club,Electronics Store,Fast Food Restaurant,Fish & Chips Shop,French Restaurant,Garden,Garden Center,Gas Station,Women's Store


In [76]:
Enfield_merged.loc[Enfield_merged['Cluster Labels'] == 4, Enfield_merged.columns[[1] + list(range(5, Enfield_merged.shape[1]))]]

,Neighborhood,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Bulls Cross,4,Trail,Soccer Field,Garden,Park,Pub,Women's Store,Garden Center,Fast Food Restaurant,Fish & Chips Shop,French Restaurant
5,Edmonton,4,Turkish Restaurant,Train Station,Bookstore,Convenience Store,Park,Coffee Shop,Music Venue,Women's Store,Gas Station,French Restaurant
7,Enfield Lock,4,Fish & Chips Shop,Train Station,Trail,Coffee Shop,Park,Women's Store,Gift Shop,French Restaurant,Garden,Garden Center
12,Hadley Wood,4,Tunnel,Trail,Mobile Phone Shop,Park,Women's Store,Gift Shop,Fish & Chips Shop,French Restaurant,Garden,Garden Center


In [77]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Enfield_merged['Latitude'], Enfield_merged['Longitude'],Enfield_merged['Neighborhood'], Enfield_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster],
        fill=True,
        fill_color=rainbow[cluster],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Conclusion: I will choose neighbourhood from the cluster 2 or cluster 4,since my priority is connectivity to work,and the most common venues shoudl be Bus stop or Trains tation which both the clusters offer.

So I will choose to stay in following neighbourhood: Edmonton,Enfield lock or Arnovs Grove,Palmers Green